# TNFR Structural Field Tetrad: End-to-End Exploration
This notebook explores the four CANONICAL TNFR structural fields (Φ_s, |∇φ|, K_φ, ξ_C) and related research utilities on multiple graph topologies, using the implementation in this repository.

Prerequisites (run in VS Code terminal, PowerShell):
- python -m pip install --upgrade numpy networkx matplotlib scikit-learn

We’ll keep dependencies light and stick to numpy, networkx, and matplotlib. If scikit-learn is missing, phase symmetry metrics will gracefully fall back.

In [1]:
# Set Up Environment
import random
import numpy as np
np.set_printoptions(precision=4, suppress=True)
random.seed(42)
np.random.seed(42)
print("Seeds set for reproducibility.")

Seeds set for reproducibility.


In [ ]:
# Configure Project Path (VS Code workspace)
import sys, os
from pathlib import Path

# Find repo root that contains src/tnfr
cwd = Path.cwd()
repo_root = None
for base in [cwd] + list(cwd.parents):
    if (base / "src" / "tnfr").exists() or (base / "pyproject.toml").exists():
        repo_root = base
        break

SRC = (repo_root / "src") if repo_root else (cwd / "src")
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))
print("Project src set:", SRC if SRC.exists() else "NOT FOUND")

In [ ]:
# Import Libraries and TNFR Field Functions
import math
import networkx as nx
import matplotlib.pyplot as plt
from tnfr.physics.fields import (
    compute_structural_potential,
    compute_phase_gradient,
    compute_phase_curvature,
    compute_k_phi_multiscale_variance,
    fit_k_phi_asymptotic_alpha,
    k_phi_multiscale_safety,
    estimate_coherence_length,
    fit_correlation_length_exponent,
    measure_phase_symmetry,
    path_integrated_gradient,
 )
print("Imports OK.")

In [ ]:
# Create Benchmark Graphs (ring, WS, scale-free, grid, tree)
def make_graphs(n=60, seed=42):
    rng = np.random.default_rng(seed)
    graphs = {}
    graphs['ring'] = nx.cycle_graph(n)
    graphs['ws'] = nx.watts_strogatz_graph(n, k=min(4, n-1), p=0.2, seed=seed)
    # Use Barabasi-Albert for scale-free (undirected)
    graphs['scale_free'] = nx.barabasi_albert_graph(n, m=2, seed=seed)
    # Grid: relabel to 0..N-1
    side = max(2, int(math.sqrt(n)))
    Ggrid = nx.grid_2d_graph(side, side)
    mapping = {node: i for i, node in enumerate(Ggrid.nodes())}
    graphs['grid'] = nx.relabel_nodes(Ggrid, mapping)
    # Tree: balanced binary with ~n nodes
    h = max(1, int(math.log2(n)))
    Gtree = nx.balanced_tree(r=2, h=h)
    if Gtree.number_of_nodes() > n:
        nodes_to_remove = list(Gtree.nodes())[n:]
        Gtree.remove_nodes_from(nodes_to_remove)
    graphs['tree'] = Gtree
    return graphs
graphs = make_graphs(n=60, seed=42)
print("Graphs:", {k: (v.number_of_nodes(), v.number_of_edges()) for k,v in graphs.items()})

In [ ]:
# Initialize Node Telemetry (θ, ΔNFR, coherence)
def init_telemetry(G, seed=123, grad_bias=0.2):
    rng = np.random.default_rng(seed)
    # random base phases in [0, 2π)
    base = rng.uniform(0.0, 2*math.pi, size=G.number_of_nodes())
    # Introduce mild structured gradient by projecting node index
    nodes = list(G.nodes())
    for idx, n in enumerate(nodes):
        theta = (base[idx] + grad_bias * (idx / max(1, len(nodes)-1))) % (2*math.pi)
        delta_nfr = float(rng.uniform(0.05, 0.2))
        G.nodes[n]['theta'] = float(theta)
        G.nodes[n]['phase'] = float(theta)  # alias
        G.nodes[n]['delta_nfr'] = delta_nfr
        G.nodes[n]['dnfr'] = delta_nfr
        G.nodes[n]['coherence'] = 1.0 / (1.0 + abs(delta_nfr))
for name, G in graphs.items():
    init_telemetry(G, seed=100 + hash(name)%1000, grad_bias=0.4)
print("Telemetry initialized: theta, delta_nfr, coherence.")

In [ ]:
# Compute Structural Potential Φ_s and drift check
def phi_s_and_drift(G):
    phi_before = compute_structural_potential(G, alpha=2.0)
    # Benign perturbation: small uniform delta_nfr tweak
    for n in G.nodes():
        G.nodes[n]['delta_nfr'] = float(G.nodes[n]['delta_nfr'] * 1.01)
        G.nodes[n]['dnfr'] = float(G.nodes[n]['dnfr'] * 1.01)
    phi_after = compute_structural_potential(G, alpha=2.0)
    nodes = list(G.nodes())
    if nodes:
        drift = float(np.mean([abs(phi_after[n] - phi_before[n]) for n in nodes]))
    else:
        drift = 0.0
    return phi_before, phi_after, drift
for name, G in graphs.items():
    phi_b, phi_a, drift = phi_s_and_drift(G)
    print(f"{name:10s} ΔΦ_s (mean drift): {drift:.4f}  (threshold 2.0)")

In [ ]:
# Compute Phase Gradient |∇φ| and flag risky nodes (|∇φ| ≥ 0.38)
grad_maps = {}
for name, G in graphs.items():
    grad = compute_phase_gradient(G)
    grad_maps[name] = grad
    vals = np.array(list(grad.values())) if grad else np.array([])
    risky = int(np.sum(vals >= 0.38)) if vals.size else 0
    print(f"{name:10s} |∇φ| mean={vals.mean():.4f} risky_nodes={risky}")

In [ ]:
# Compute Phase Curvature K_φ and hotspots (|K_φ| ≥ 3.0)
kphi_maps = {}
for name, G in graphs.items():
    kphi = compute_phase_curvature(G)
    kphi_maps[name] = kphi
    absmax = float(np.max(np.abs(list(kphi.values())))) if kphi else 0.0
    hotspots = sum(1 for v in kphi.values() if abs(v) >= 3.0) if kphi else 0
    print(f"{name:10s} |K_φ|_max={absmax:.3f} hotspots={hotspots}")

In [ ]:
# Multiscale K_φ Variance and α fit
for name, G in graphs.items():
    var_by_scale = compute_k_phi_multiscale_variance(G, scales=(1,2,3,5))
    fit = fit_k_phi_asymptotic_alpha(var_by_scale)
    print(f"{name:10s} scales={list(var_by_scale.keys())} α={fit['alpha']:.3f} R²={fit['r_squared']:.3f} ({fit['fit_quality']})")

In [ ]:
# Multiscale Curvature Safety Check (α_hint=2.76, tolerance=2.0)
for name, G in graphs.items():
    safety = k_phi_multiscale_safety(G, scales=(1,2,3,5), alpha_hint=2.76, tolerance_factor=2.0, fit_min_r2=0.5)
    print(f"{name:10s} safe={safety['safe']} violations={safety['violations']} R²={safety['fit'].get('r_squared',0.0):.3f}")

In [ ]:
# Estimate Coherence Length ξ_C via Radial Decay
for name, G in graphs.items():
    xi = estimate_coherence_length(G, coherence_key='coherence')
    print(f"{name:10s} ξ_C ≈ {xi:.3f}")

In [ ]:
# Sweep intensity I around I_c and fit ν for ξ_C ~ |I - I_c|^{-ν}
I_c = 2.015
rng = np.random.default_rng(7)
Is = np.array([1.6, 1.8, 1.9, 2.0, 2.03, 2.1, 2.3, 2.5], dtype=float)
xi_vals = []
# Use the WS graph as representative for the sweep
G_ws = graphs['ws'].copy()
for I in Is:
    # Perturb ΔNFR magnitude to emulate regime intensity
    scale = 1.0 + (I - I_c) * 0.5
    for n in G_ws.nodes():
        base = float(G_ws.nodes[n]['delta_nfr'])
        G_ws.nodes[n]['delta_nfr'] = abs(base * scale)
        G_ws.nodes[n]['dnfr'] = abs(base * scale)
    xi_vals.append(estimate_coherence_length(G_ws, coherence_key='coherence'))
xi_vals = np.array(xi_vals, dtype=float)
fit_res = fit_correlation_length_exponent(Is, xi_vals, I_c=I_c, min_distance=0.01)
print("I:", Is)
print("xi_C:", xi_vals)
print({k: v for k,v in fit_res.items() if not k.startswith('n_points')})

In [ ]:
# Phase Symmetry Metrics (clustering on unit circle)
for name, G in graphs.items():
    sym = measure_phase_symmetry(G)
    print(f"{name:10s}", sym)

In [ ]:
# Path-Integrated Gradient (PIG) on a shortest path
for name, G in graphs.items():
    nodes = list(G.nodes())
    if len(nodes) < 2:
        continue
    src, dst = nodes[0], nodes[-1]
    try:
        path = nx.shortest_path(G, src, dst)
    except Exception:
        continue
    pig = path_integrated_gradient(G, path)
    print(f"{name:10s} path length {len(path)} PIG={pig:.3f}")

In [ ]:
# Minimal Assertions (sanity checks, soft)
issues = []
for name, G in graphs.items():
    grad = grad_maps[name]
    kphi = kphi_maps[name]
    # Shapes match |V|
    if len(grad) != G.number_of_nodes() or len(kphi) != G.number_of_nodes():
        issues.append(f"{name}: length mismatch (grad {len(grad)}, kphi {len(kphi)}, nodes {G.number_of_nodes()})")
    # Non-trivial values
    try:
        _max_abs = float(np.max(np.abs(list(kphi.values())))) if kphi else 0.0
    except Exception as e:
        issues.append(f"{name}: kphi values error: {e}")
        _max_abs = 0.0
    # Multiscale variance should generally not explode with r in healthy-ish setup
    var_by_scale = compute_k_phi_multiscale_variance(G, scales=(1,2,3))
    vals = [var_by_scale[r] for r in sorted(var_by_scale.keys())]
    if max(vals) > 10 * (min(vals) + 1e-9):
        issues.append(f"{name}: k_phi variance grows too fast across scales: {vals}")
    safety = k_phi_multiscale_safety(G, scales=(1,2,3,5), alpha_hint=2.76)
    if not isinstance(safety.get('safe', None), bool):
        issues.append(f"{name}: safety result malformed: {safety}")

if issues:
    print("WARN: Sanity checks found issues:")
    for it in issues:
        print(" -", it)
else:
    print("Assertions passed.")

In [ ]:
# Optional Plots and Diagnostics
def _tight():
    plt.tight_layout()
    plt.show()

# Histograms of |∇φ| and K_φ for one graph
name = 'ws' if 'ws' in graphs else list(graphs.keys())[0]
G = graphs[name]
grad = np.array(list(grad_maps[name].values()))
kphi = np.array(list(kphi_maps[name].values()))
fig, axs = plt.subplots(1, 2, figsize=(10,4))
axs[0].hist(grad, bins=20, color='tab:blue', alpha=0.8)
axs[0].set_title(f"{name} |∇φ| histogram")
axs[1].hist(kphi, bins=20, color='tab:orange', alpha=0.8)
axs[1].set_title(f"{name} K_φ histogram")
_tight()

# Log–log var(K_φ) vs r with fitted line
var_by_scale = compute_k_phi_multiscale_variance(G, scales=(1,2,3,5))
fit = fit_k_phi_asymptotic_alpha(var_by_scale)
rs = np.array(sorted(var_by_scale.keys()), dtype=float)
vars_ = np.array([var_by_scale[int(r)] for r in rs], dtype=float)
fig, ax = plt.subplots(1, 1, figsize=(5,4))
ax.plot(np.log(rs+1e-9), np.log(vars_+1e-12), 'o-', label='data')
if fit['alpha'] > 0:
    # Fit line: log(var) ~ a - alpha * log(r) (we'll reconstruct via simple linear fit)
    # Refit here to get intercept for plotting
    X = np.vstack([np.ones_like(np.log(rs)), -np.log(rs)]).T
    y = np.log(vars_ + 1e-12)
    coeffs, *_ = np.linalg.lstsq(X, y, rcond=None)
    y_pred = X @ coeffs
    ax.plot(np.log(rs+1e-9), y_pred, '-', label=f"fit α≈{fit['alpha']:.2f}, R²≈{fit['r_squared']:.2f}")
ax.set_xlabel('log r')
ax.set_ylabel('log var(K_φ)')
ax.legend()
_tight()